# 02_03 CNN

In [1]:
import pandas as pd
import numpy as np
from tensorflow.contrib import keras

from smh_eyetracking.features02 import config as config_features02
from smh_eyetracking.features02.utils.features02_dlib import FEATURES, TARGETS
from smh_eyetracking.keras import config as config_keras
from smh_eyetracking.keras import losses
from smh_eyetracking.utils import data_model

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_CSV,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    validation_size=0.15,
    test_size=0.15
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [2]:
img_width, img_height = config_features02.FEATURES02_EYES_SIZE
img_shape = (img_height, img_width)

def get_model():

    # Inputs
    left_imgs = keras.layers.Input(shape=img_shape, name='left_imgs', dtype='float32')
    right_imgs = keras.layers.Input(shape=img_shape, name='right_imgs', dtype='float32')
    features = keras.layers.Input(shape=(len(FEATURES),), name='features', dtype='float32')
    # Features
    f1 = keras.layers.Dense(256, activation=keras.activations.relu)(features)
    f2 = keras.layers.Dense(128, activation=keras.activations.relu)(f1)
    # Left imgs
    l1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(left_imgs)
    )
    l2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same')(l1)
    # Right imgs
    r1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(right_imgs)
    )
    r2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same', input_shape=(24, 32, 1))(r1)
    # Concatenate
    concat = keras.layers.Concatenate()([
        keras.layers.Flatten()(l2),
        keras.layers.Flatten()(r2),
        f2
    ])
    # Dense
    d_1 = keras.layers.Dense(256, activation=keras.activations.relu)(concat)
    d_2 = keras.layers.Dense(128, activation=keras.activations.relu)(d_1)
    d_3 = keras.layers.Dense(32, activation=keras.activations.relu)(d_2)
    d_4 = keras.layers.Dense(2)(d_3)
    # Model
    model = keras.models.Model(
        inputs=[left_imgs, right_imgs, features],
        outputs=[d_4]
    )
    return model

### Parameters

In [3]:
MODEL_NAME = '03_CNN-09'

EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
DECAY = 0.000001


LOSS = losses.mean_euclidean

In [4]:
model = get_model()


model.compile(
    loss=LOSS,
    metrics=[losses.mean_euclidean],
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=DECAY)
)

print("Parameters to adjust: {}".format(
    np.sum([keras.backend.count_params(p) for p in set(model.trainable_weights)])
))

Parameters to adjust: 1734242


### Train

In [ ]:
model.fit(
    x={
        'left_imgs':train_imgs_left,
        'right_imgs': train_imgs_right,
        'features': train_data[FEATURES].as_matrix()
    },
    y=train_data[TARGETS].as_matrix(),
    validation_data=(
        {
            'left_imgs': validation_imgs_left,
            'right_imgs': validation_imgs_right,
            'features': validation_data[FEATURES].as_matrix()
        },
        validation_data[TARGETS].as_matrix()
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1, callbacks=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0
)

In [ ]:
model.save(config_keras.PATH_MODELS_KERAS+MODEL_NAME)

### Test

In [ ]:
model_test = keras.models.load_model(
    filepath=config_keras.PATH_MODELS_KERAS+MODEL_NAME,
    custom_objects={
        "mean_euclidean": losses.mean_euclidean,
        "ms_euclidean": losses.ms_euclidean,
        "reg_mean_euclidean": losses.reg_mean_euclidean
    }
)

In [ ]:
model_test.evaluate(
    x={
        'left_imgs':test_imgs_left,
        'right_imgs': test_imgs_right,
        'features': test_data[FEATURES].as_matrix()
    },
    y=test_data[TARGETS].as_matrix(),
    batch_size=1,
    verbose=1, sample_weight=None
)

### Results

Corrected dataset


| Name | Epochs | Batch Size | Learning rate | Decay | Loss | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| 03_CNN-01 | 50 | 128| 0.005 | 0.0001 | mean_euclidean | 0.1385 | 0.2369 | 0.2481 |
| 03_CNN-02 | 50 | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0394 | 0.1696 | 0.1768 |
| 03_CNN-03 | 117 (stopped) | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0232 | 0.1705 | 0.1771 |
| 03_CNN-04 | 50 | 128 | 0.00005 | 0.000001 | mean_euclidean | 0.0861 | 0.1644 | 0.1801 |
| 03_CNN-05 | 50 | 64 | 0.00005 | 0.000001 | mean_euclidean | 0.0565 | 0.1586 | 0.1742 |
| 03_CNN-06 | 50 | 32 | 0.00005 | 0.000001 | mean_euclidean | 0.0441 | 0.1670 | 0.1808 |
| 03_CNN-07 | 50 | 64 | 0.000005 | 0.000001 | mean_euclidean | 0.1824 | 0.1902 | 0.1979 |
| 03_CNN-08 | 100 | 64 | 0.000005 | 0.000001 | mean_euclidean | 0.1411 | 0.1745 | 0.1892 |
| 03_CNN-09 | 50 | 64 | 0.00001 | 0.000001 | mean_euclidean | 0.1342 | 0.1742 | 0.1910 |
| 03_CNN-10 | 100 | 64 | 0.00001 | 0.000001 | mean_euclidean | 0.0935 | 0.1658 | 0.1781 |
| 03_CNN-11 | 150 | 64 | 0.00001 | 0.000001 | mean_euclidean | 0.0702 | 0.1647 | 0.1746 |
| 03_CNN-12 | 200 | 64 | 0.00001 | 0.000001 | mean_euclidean | 0.0554 | 0.1658 | 0.1747 |